In [1]:
!pip install py-entitymatching

     |████████████████████████████████| 2.0MB 3.9MB/s 
     |████████████████████████████████| 788kB 41.4MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 
     |████████████████████████████████| 368kB 21.8MB/s 
     |████████████████████████████████| 665kB 34.3MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634152 sha256=287d2a934c565ec95e1cfb381ebbcdfa4da3d0166698c0eb57d4b3a014bce9a2
  Stored in directory: /root/.cache/pip/wheels/88/65/50/a8bc0f26e8a82c931658153c4f37cbf5debc22cb0f61dbcd05
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3723773 sha256=4bf0b34ebc1b1f9ba2ac754771823fdbe766a0fd512b1d742bd2e0cdf7682b0f
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055262 sha256=926e880fa47c050e8ca

In [1]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [46]:
path_variable = "/content/gdrive/My Drive/BTP/Code/Dataset/Music/"

In [25]:
import py_entitymatching as em
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = path_variable+"music1.csv"
path_B = path_variable+"music2.csv"
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0
Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [17]:
# Displaying first two entries from the first music dataset
A.head(2)

,Sno,Album_Name,Album_Price,Artist_Name,CopyRight,Customer_Rating,Genre,Price,Released,Song_Name,Time,Group
0,1,Welcome to Cam Country - EP,$4.26,Cam,2015 Sony Music Entertainment,4.72396,"Country,Music,Contemporary Country,Honky Tonk",$0.99,31-Mar-15,Runaway Train,3:01,Country
1,2,Me 4 U,$9.99,Omi,"2015 Ultra Records, LLC under exclusive license to Columbia Records, a Division of Sony Music E...",3.38158,"Pop/Rock,Music,Pop,Dance,R&B/Soul",Album Only,NaN,Track 14,3:41,Pop


In [18]:
# Displaying first two entries from the second music dataset
B.head(-2)

,Sno,Album_Name,Artist_Name,Song_Name,Price,Time,Released,Label,Copyright,Genre,Group
0,1,! (Volume 2) [Explicit],Rusko,Saxophone Stomp [Explicit],$1.29,3:20,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
1,2,! (Volume 2) [Explicit],Rusko,I Wanna Mingle [feat. Pusher],$1.29,2:36,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
2,3,! (Volume 2) [Explicit],Rusko,Lytah,$1.29,3:48,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
3,4,! (Volume 2) [Explicit],Rusko,Slappy Pappy,$1.29,3:55,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
4,5,! (Volume 2) [Explicit],Rusko,Fushion Jam,$1.29,1:48,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
...,...,...,...,...,...,...,...,...,...,...,...
55916,55953,x,Ed Sheeran,Nina,$1.29,3:45,"June 20, 2014",Atlantic Records UK,(C) 2014 mau5trap Recordings Ltd,Pop,Pop
55917,55954,x,Ed Sheeran,Photograph,$1.29,4:19,"June 20, 2014",Atlantic Records UK,(C) 2014 mau5trap Recordings Ltd,Pop,Pop
55918,55955,x,Ed Sheeran,Bloodstream,$1.29,5:00,"June 20, 2014",Atlantic Records UK,(C) 2014 mau5trap Recordings Ltd,Pop,Pop
55919,55956,x,Ed Sheeran,Tenerife Sea,$1.29,4:01,"June 20, 2014",Atlantic Records UK,(C) 2014 mau5trap Recordings Ltd,Pop,Pop


**Blocking**

In [26]:
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, "Album_Name", "Album_Name", overlap_size=2, 
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","CopyRight", "Released", "Price" ,"Time","Group"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","Copyright", "Released", "Price" ,"Time","Group"]
                    )


# Printing length of candidate set
print(len(C))
print(C.head(2))

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


3042443
   _id  ltable_Sno  rtable_Sno  ... rtable_Price rtable_Time rtable_Group
0    0        3996           1  ...        $1.29        3:20        Dance
1    1        3996           2  ...        $1.29        2:36        Dance

[2 rows x 21 columns]


In [27]:
# Overlap based blocker, which blocks on the basis of overlap of some qgram between two strings
C = ob.block_candset(C,"Song_Name", "Song_Name",overlap_size=2)
# Printing length of candidate set
print(len(C))
print(C.head(2))

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:48


63246
    _id  ltable_Sno  rtable_Sno  ... rtable_Price rtable_Time rtable_Group
9     9         726          28  ...        $0.99        8:49   Electronic
10   10         727          28  ...        $0.99        8:49   Electronic

[2 rows x 21 columns]


In [29]:
C = ob.block_candset(C, 'CopyRight', 'Copyright',overlap_size=2)
# Printing length of candidate set
print(len(C))
print(C.head(2))

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


12829
       _id  ltable_Sno  rtable_Sno  ... rtable_Price rtable_Time rtable_Group
3173  3173        2584         229  ...        $1.29        3:03        Dance
3210  3210        2004         230  ...        $1.29        3:52   Electronic

[2 rows x 21 columns]


**Feature Generation**

In [30]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)
# List the names of the features generated
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [31]:
from math import *

def jaccard_sim(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)
# Manually creating features for some attribute

def time_time_lev_sim(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return em.lev_sim(p1,p2)

def time_time_cos(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return em.cosine(list(p1),list(p2))

def time_time_jac(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return jaccard_sim(p1,p2)

def time_time_mel(ltuple,rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    if em.monge_elkan(list(p1),list(p2))=="nan":
        return 0
    else:
        return em.monge_elkan(list(p1),list(p2))

def time_time_sw(ltuple,rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    if em.smith_waterman(p1,p2)=="nan":
        return 0
    else:
        return em.smith_waterman(p1,p2)
    
def released_released_lev_sim(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return em.lev_sim(s1,p2)

def released_released_cos(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return em.cosine(list(s1),list(p2))

def released_released_jac(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return jaccard_sim(s1,p2)

def released_released_mel(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    if em.monge_elkan(list(s1),list(p2))=="nan":
        return 0
    else:
        return em.monge_elkan(list(s1),list(p2))

def released_released_sw(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    if em.smith_waterman(s1,p2)=="nan":
        return 0
    else:
        return em.smith_waterman(s1,p2)

def copyright_copyright_lev_sim(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return em.lev_sim(p1,p2)

def copyright_copyright_cos(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return em.cosine(list(p1),list(p2))

def copyright_copyright_jac(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return jaccard_sim(p1,p2)

def copyright_copyright_mel(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    if em.monge_elkan(list(p1),list(p2))=="nan":
        return 0
    else:
        return em.monge_elkan(list(p1),list(p2))

def copyright_copyright_sw(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    if em.smith_waterman(p1,p2)=="nan":
        return 0
    else:
        return em.smith_waterman(p1,p2)

def price_price_lev_sim(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return em.lev_sim(p1,p2)

def price_price_cos(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return em.cosine(list(p1),list(p2))

def price_price_jac(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return jaccard_sim(p1,p2)

def price_price_mel(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    if em.monge_elkan(list(p1),list(p2))=="nan":
        return 0
    else:
        return em.monge_elkan(list(p1),list(p2))

def price_price_sw(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    if em.smith_waterman(p1,p2)=="nan":
        return 0
    else:
        return em.smith_waterman(p1,p2)

In [32]:
em.add_blackbox_feature(feature_table, 'time_time_lev_sim', time_time_lev_sim)
em.add_blackbox_feature(feature_table, 'time_time_cos', time_time_cos)
em.add_blackbox_feature(feature_table, 'time_time_jac', time_time_jac)
em.add_blackbox_feature(feature_table, 'time_time_mel', time_time_mel)
# em.add_blackbox_feature(feature_table, 'time_time_sw', time_time_sw)

em.add_blackbox_feature(feature_table, 'released_released_lev_sim', released_released_lev_sim)
em.add_blackbox_feature(feature_table, 'released_released_cos', released_released_cos)
em.add_blackbox_feature(feature_table, 'released_released_jac', released_released_jac)
em.add_blackbox_feature(feature_table, 'released_released_mel', released_released_mel)
# em.add_blackbox_feature(feature_table, 'released_released_sw', released_released_sw)

em.add_blackbox_feature(feature_table, 'copyright_copyright_lev_sim', copyright_copyright_lev_sim)
em.add_blackbox_feature(feature_table, 'copyright_copyright_cos', copyright_copyright_cos)
em.add_blackbox_feature(feature_table, 'copyright_copyright_jac', copyright_copyright_jac)
em.add_blackbox_feature(feature_table, 'copyright_copyright_mel', copyright_copyright_mel)
# em.add_blackbox_feature(feature_table, 'copyright_copyright_sw', copyright_copyright_sw)

# em.add_blackbox_feature(feature_table, 'price_price_lev_sim', price_price_lev_sim)
# em.add_blackbox_feature(feature_table, 'price_price_cos', price_price_cos)
# em.add_blackbox_feature(feature_table, 'price_price_jac', price_price_jac)
# em.add_blackbox_feature(feature_table, 'price_price_mel', price_price_mel)
# em.add_blackbox_feature(feature_table, 'price_price_sw', price_price_sw)

True

In [33]:
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [34]:
# Only take features corresponding to the three sim scores : cosine, Jaccard, lev_sim
feature_table = feature_table.drop([0,1,2,3,6,8,10,11,14,16,18,19,22,24,26,27,30,32,34,35,36,37,38,39,40,41],axis=0)
# List the names of the features generated
feature_table['feature_name']

In [36]:
H = em.extract_feature_vecs(C, feature_table=feature_table)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:15:17


,_id,ltable_Sno,rtable_Sno,Album_Name_Album_Name_jac_qgm_3_qgm_3,Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0,Album_Name_Album_Name_mel,Album_Name_Album_Name_lev_sim,Artist_Name_Artist_Name_jac_qgm_3_qgm_3,Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0,Artist_Name_Artist_Name_mel,Artist_Name_Artist_Name_lev_sim,Genre_Genre_jac_qgm_3_qgm_3,Genre_Genre_cos_dlm_dc0_dlm_dc0,Genre_Genre_mel,Genre_Genre_lev_sim,Song_Name_Song_Name_jac_qgm_3_qgm_3,Song_Name_Song_Name_cos_dlm_dc0_dlm_dc0,Song_Name_Song_Name_mel,Song_Name_Song_Name_lev_sim,time_time_lev_sim,time_time_cos,time_time_jac,time_time_mel,released_released_lev_sim,released_released_cos,released_released_jac,released_released_mel,copyright_copyright_lev_sim,copyright_copyright_cos,copyright_copyright_jac,copyright_copyright_mel
3173,3173,2584,229,0.413043,0.516398,0.791258,0.600000,1.0,1.0,1.000000,1.000000,0.517241,0.0,0.836364,0.681818,0.600000,0.666667,0.846642,0.807692,1.00,1.00000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,0.960784,0.957427,0.916667,1.000000
3210,3210,2004,230,0.341463,0.516398,0.751534,0.533333,1.0,1.0,1.000000,1.000000,0.424242,0.0,0.850303,0.433333,0.195652,0.250000,0.702469,0.444444,0.50,0.75000,0.600000,0.75,1.000000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.000000
3211,3211,2575,230,0.341463,0.516398,0.751534,0.533333,1.0,1.0,1.000000,1.000000,0.424242,0.0,0.850303,0.433333,0.195652,0.250000,0.702469,0.444444,0.50,0.75000,0.600000,0.75,1.000000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.000000
3212,3212,2576,230,0.341463,0.516398,0.751534,0.533333,1.0,1.0,1.000000,1.000000,0.424242,0.0,0.850303,0.433333,0.195652,0.250000,0.702469,0.444444,0.50,0.75000,0.600000,0.75,1.000000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.000000
3295,3295,2004,231,0.341463,0.516398,0.751534,0.533333,1.0,1.0,1.000000,1.000000,0.424242,0.0,0.850303,0.433333,0.219512,0.288675,0.707148,0.458333,0.50,0.57735,0.400000,0.50,1.000000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015737,3015737,1412,55461,0.102362,0.084515,0.592127,0.298851,0.0,0.0,0.615591,0.096774,0.000000,0.0,0.522348,0.125000,0.110169,0.094491,0.579829,0.287356,0.50,0.50000,0.333333,0.50,0.470588,0.483494,0.315789,0.642857,0.184000,0.633866,0.416667,0.958333
3015749,3015749,2589,55461,0.080000,0.077152,0.585983,0.195402,0.0,0.0,0.469459,0.161290,0.000000,0.0,0.496212,0.181818,0.086207,0.084515,0.559600,0.206897,0.50,0.57735,0.400000,0.75,0.411765,0.617213,0.444444,0.714286,0.408000,0.706188,0.545455,0.887755
3015750,3015750,3518,55461,0.073171,0.000000,0.597166,0.206897,0.0,0.0,0.459677,0.129032,0.000000,0.0,0.411765,0.000000,0.078947,0.000000,0.589783,0.275862,0.75,0.75000,0.600000,0.75,0.631579,0.741249,0.588235,0.842105,0.218750,0.789330,0.650000,0.945312
3015751,3015751,4466,55461,0.068376,0.089087,0.622992,0.218391,0.0,0.0,0.541150,0.129032,0.133333,0.0,0.489881,0.312500,0.084112,0.101015,0.616242,0.218391,0.50,0.50000,0.333333,0.50,0.823529,0.889499,0.800000,0.937500,0.392000,0.762443,0.605263,0.926829


In [50]:
import pickle

dbfile = open(path_variable+'files/feature_matrix', 'wb') 
pickle.dump(H, dbfile) 
dbfile.close()

candidate_set = open(path_variable+'files/C', 'wb') 
pickle.dump(C, candidate_set)
candidate_set.close()